## Análisis de Documentos y Visualización de Embeddings

En este notebook, procesamos los manuales de Sabentis para generar embeddings utilizando un modelo BERT preentrenado específico para el idioma español. Posteriormente, aplicamos técnicas avanzadas de visualización de datos como PCA, t-SNE y UMAP para representar gráficamente los resultados. Este análisis nos permite explorar la estructura y similitud de los documentos procesados.  

El proceso consisste en cargar varios manuales relacionados con distintos aspectos de la plataforma Sabentis, incluyendo auditorías, ausentismo, estructura organizativa, identificación y evaluación de riesgos, gestión de información y planes de emergencia.
Los documentos se dividen en fragmentos (chunks) de tamaño 500 palabras con un solapamiento de 400 palabras para asegurar una cobertura completa y contextual de los textos.

Generamos los embelding utilizando un modelo BERT preentrenado en español para generar embeddings de los fragmentos de texto.

Y para visualizar aplicamos técnicas de reducción de dimensionalidad como PCA, t-SNE y UMAP para reducir la alta dimensionalidad de los embeddings a espacios bidimensionales o tridimensionales.  

Para evaluar la capacidad de los manuales en responder a consultas específicas, hemos formulado un conjunto de preguntas que se dividen en tres categorías:

1.Preguntas Respondidas por los Manuales.
2.Preguntas No Respondidas por los Manuales
3.Pregunta Fuera de Contexto

El objetivo principal de este notebook es analizar cómo los manuales de Sabentis pueden proporcionar respuestas a preguntas específicas y evaluar la capacidad del modelo BERT para capturar la semántica de los textos. Al aplicar técnicas de visualización de datos, buscamos identificar la estructura subyacente de los documentos y explorar cómo se relacionan entre sí.

### 1. Importar Librerías

In [1]:
# Librerías para el manejo de archivos y operaciones numéricas
import os
import numpy as np

# Librerías para visualización
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Librerías para procesamiento de texto y modelos de lenguaje
from transformers import BertTokenizer, BertModel

# Librerías para reducción de dimensionalidad y métricas
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
##import umap
from sklearn.metrics.pairwise import cosine_similarity

/opt/homebrew/Caskroom/miniconda/base/envs/tfm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. Inicializar Tokenizer y Modelo BERT

Se carga un modelo BERT preentrenado específico para el idioma español.

In [2]:
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')
model = BertModel.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 3. Función para Obtener Embeddings BERT

**bert_embeddings:** Toma un texto como entrada y devuelve los embeddings BERT.  
**tokenizer:** Convierte el texto en tensores adecuados para el modelo.  
**model:** Obtiene los embeddings del modelo BERT.

In [3]:
def bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].detach().numpy()

### 4. Función para Dividir Texto en Chunks

**chunk_text:** Divide el contenido de un archivo de texto en fragmentos (chunks) de tamaño especificado con cierta superposición.  
**chunk_size:** Tamaño de cada chunk.  
**overlap:** Superposición entre chunks consecutivos.

### 5. Rutas de Archivos de Manuales

Diccionario con nombres de manuales y sus respectivas rutas de archivo.

In [4]:
file_paths = {
    "Auditorias": "../chunks/auditorias.txt",
    "Ausentismo": "../chunks/ausentismo.txt",
    "Estructura Organizativa": "../chunks/estructura organizativa.txt",
    "Riesgos": "../chunks/riesgos.txt",
    "Información": "../chunks/información.txt",
    "Emergencia": "../chunks/emergencia.txt"
}

### 6. Dividir Manuales en Chunks

Para cada manual, se lee y se divide el contenido en chunks utilizando la función chunk_text.

### 7. Unir Todos los Chunks y Sus Etiquetas

**all_chunks:** Lista que contiene todos los chunks de todos los manuales.  
**chunk_labels:** Lista de etiquetas que indica a qué manual pertenece cada chunk.

In [7]:
chunks = {}

for key, path in file_paths.items():
    with open(path, 'r') as file:
        chunks[key] = file.read().split('\n\n')

all_chunks = []
chunk_labels = []
for manual, manual_chunks in chunks.items():
    all_chunks.extend(manual_chunks)
    chunk_labels.extend([manual] * len(manual_chunks))

### 8. Obtener Embeddings BERT para Todos los Chunks

Se generan los embeddings BERT para cada chunk y se almacenan en una matriz chunk_embeddings.

In [8]:
chunk_embeddings = np.array([bert_embeddings(chunk)[0] for chunk in all_chunks])

### 9. Lista de Preguntas

Lista de preguntas a analizar, algunas relevantes a los manuales y otras no.

In [9]:
questions = [
    # 7 preguntas con la respuesta en los manuales
    "¿Cómo se diferencia la ubicación física de la operativa?",  # Manual Estructura Organizativa pág. 60 y 73.
    "¿Qué información se incluye en el informe de la evaluación de riesgos que ofrece la plataforma y cuál es su utilidad para la gestión de riesgos en la empresa?",  # Manual Identificación y Evaluación de Riesgos (IER); pág. 24 - 26.
    "¿Hay indicadores relativos al cumplimiento de normas?",  # Manual Auditorías pág. 16 17 30 y 31.
    "¿Qué papel juega la estructura organizativa en la funcionalidad general de la plataforma y cómo interactúa con otros módulos?",  # Manual Estructura Organizativa
    "¿Qué permite realizar la evaluación de riesgos?",  # Manual Identificación y Evaluación de Riesgos (IER)
    "¿Qué tipo de documentos se pueden almacenar y compartir en el espacio denominado 'Documentos' y cuál es su importancia dentro del contexto de la gestión empresarial en la plataforma?",  # Manual del Repositorio documental
    "¿Se pueden llevar a cabo auditorías internas?",  # Manual Auditorías

    # 2 preguntas que no se puedan resolver con los manuales
    "¿Cuál es la tasa de adopción de esta plataforma en el mercado?",
    "¿Cómo se compara esta plataforma con otras soluciones de gestión de SST?",

    # 1 pregunta totalmente fuera del contexto
    "¿Qué día hará mañana?"
]

### 10. Obtener Embeddings BERT para las Preguntas

Generar embeddings BERT para cada pregunta y almacenarlos en una matriz question_embeddings.

In [10]:
# Obtener embeddings para todos los chunks
embeddings = np.vstack([bert_embeddings(chunk) for chunk in all_chunks])

In [11]:
question_embeddings = np.array([bert_embeddings(question)[0] for question in questions])

### 12. Reducción de Dimensionalidad a 2 Componentes con PCA y TSNE

**PCA:** Reducción de dimensionalidad a 2 componentes para los embeddings de los chunks y las preguntas.  
**TSNE:** Otra técnica de reducción de dimensionalidad a 2 componentes.

In [12]:
pca = PCA(n_components=2)
pca_result = pca.fit_transform(chunk_embeddings)
question_pca_embeddings = pca.transform(question_embeddings)

tsne = TSNE(n_components=2, perplexity=40, n_iter=300)
tsne_result = tsne.fit_transform(chunk_embeddings)
question_tsne_results = tsne.fit_transform(np.vstack([chunk_embeddings, question_embeddings]))[-len(questions):]

### 13. Reducción de Dimensionalidad a 3 Componentes con PCA y TSNE

**PCA y TSNE:** También se realiza la reducción de dimensionalidad a 3 componentes para una visualización tridimensional.

In [13]:
pca_3d = PCA(n_components=3)
pca_3d_result = pca_3d.fit_transform(chunk_embeddings)
question_pca_3d_embeddings = pca_3d.transform(question_embeddings)

tsne_3d = TSNE(n_components=3, perplexity=40, n_iter=300)
tsne_3d_result = tsne_3d.fit_transform(chunk_embeddings)
question_tsne_3d_embeddings = tsne_3d.fit_transform(np.vstack([chunk_embeddings, question_embeddings]))[-len(questions):]

### 14. Función para Encontrar los Chunks Más Similares a una Pregunta

**find_top_n_chunks:** Calcula la similitud coseno entre el embedding de una pregunta y los embeddings de los chunks, ordenando los chunks por similitud.

He substituido **cosine_similarity** por una función que calcule la distancia euclidiana. La función **np.linalg.norm** se utiliza para calcular la distancia euclidiana entre dos vectores.

In [14]:

def find_top_n_chunks(question_embedding, chunk_embeddings, n=10):
    distances = np.linalg.norm(chunk_embeddings - question_embedding, axis=1)
    sorted_indices = distances.argsort()  # Ordenar de menor a mayor
    top_n_indices = sorted_indices[:n]  # Obtener los primeros n índices
    top_n_distances = distances[top_n_indices]  # Obtener las distancias correspondientes
    return top_n_indices, top_n_distances


#### Comprobación de los embeddings  

 A continuación vamos a comparar las distancias entre embeddings de frases similares y distintas.

In [15]:
# Definir tres frases: dos similares y una distinta
phrase_1 = "El sol brilla intensamente en el cielo azul."
phrase_2 = "El cielo está despejado y el sol brilla."
phrase_3 = "El coche rojo está estacionado en la calle."

# Tokenizar y obtener embeddings para cada frase
phrases = [phrase_1, phrase_2, phrase_3]
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_embedding(phrase):
    inputs = tokenizer(phrase, return_tensors='pt')
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

embeddings = [get_embedding(phrase) for phrase in phrases]

# Calcular distancias entre embeddings
dist_12 = np.linalg.norm(embeddings[0] - embeddings[1])
dist_13 = np.linalg.norm(embeddings[0] - embeddings[2])
dist_23 = np.linalg.norm(embeddings[1] - embeddings[2])

print(f"Distancia entre frases similares (1 y 2): {dist_12}")
print(f"Distancia entre frase distinta (1 y 3): {dist_13}")
print(f"Distancia entre frase distinta (2 y 3): {dist_23}")


Distancia entre frases similares (1 y 2): 4.958995819091797
Distancia entre frase distinta (1 y 3): 5.729469299316406
Distancia entre frase distinta (2 y 3): 3.9385976791381836


### 15. Evaluación de Preguntas y Búsqueda de Chunks Más Similares

Se itera sobre cada pregunta, obteniendo los embeddings y buscando los chunks más similares.  
**
similarities**: Lista para almacenar las similitudes de las preguntas con los chunks  

Se imprime cada pregunta con los chunks más similares, incluyendo el rango y la similitud.

In [16]:
similarities = []
for question_index, question in enumerate(questions):
    question_embedding = question_embeddings[question_index]
    top_n_indices, sims = find_top_n_chunks(question_embedding, chunk_embeddings, n=10)
    similarities.append(sims)
    print(f"Pregunta: {question}")
    for rank, idx in enumerate(top_n_indices):
        if idx < len(all_chunks):
            print(f"Rango: {rank+1}, Similaridad: {sims[rank]}")
            print(f"Chunk: {all_chunks[idx]}\n")

Pregunta: ¿Cómo se diferencia la ubicación física de la operativa?
Rango: 1, Similaridad: 17.050920486450195
Chunk: los trabajadores”, “Ver  puestos de trabajo”, “Exportar el cargo como plantilla” y “Aplicar plantilla”.

Rango: 2, Similaridad: 17.581830978393555
Chunk: El concepto de las herencias de los riesgos hace referencia a los riesgos a los que un trabajador  está expuesto con motivo de las relaciones que tiene con las distintas ubicaciones de la  empresa (ya sean ubicaciones físicas u operativas).     Las herencias funcionan de manera distinta dependiendo de si provienen de ubicaciones físicas  u operativas. En el primer caso, las herencias siguen una dirección de “arriba a abajo”. Es  decir, de los niveles más altos de la estructura organizativa

Rango: 3, Similaridad: 17.84666633605957
Chunk: de tener  una evaluación realizada ésta se perderá.  •  Evaluar: Permite acceder al cuestionario de evaluación.

Rango: 4, Similaridad: 18.109411239624023
Chunk: La pestaña evaluación de

### 16.Visualización de datos

Estos gráficos interactivos permiten explorar visualmente la relación entre los embeddings de los chunks y las preguntas, proporcionando una forma intuitiva de analizar los datos.

### Reducción de dimensionalidad con PCA y generación de gráfico 2D

In [37]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Función para añadir saltos de línea
def add_line_breaks(text, max_line_length=80):
    words = text.split()
    lines = []
    current_line = []
    current_length = 0
    for word in words:
        if current_length + len(word) + 1 > max_line_length:
            lines.append(' '.join(current_line))
            current_line = [word]
            current_length = len(word)
        else:
            current_line.append(word)
            current_length += len(word) + 1
    lines.append(' '.join(current_line))
    return '<br>'.join(lines)

# Preparar los datos para el gráfico 2D PCA
pca_df = pd.DataFrame(pca_result, columns=['PCA1', 'PCA2'])
pca_df['manual'] = chunk_labels
pca_df['type'] = 'chunk'
pca_df['text'] = all_chunks  # Añadir el texto del chunk para el hover

# Crear etiquetas para las preguntas
question_labels = [f"Q{i+1}" for i in range(len(questions))]

# Agregar las preguntas al dataframe
question_pca_df = pd.DataFrame(question_pca_embeddings, columns=['PCA1', 'PCA2'])
question_pca_df['manual'] = question_labels
question_pca_df['type'] = 'question'
question_pca_df['text'] = questions  # Añadir la pregunta para el hover

# Combinar ambos dataframes
combined_pca_df = pd.concat([pca_df, question_pca_df])

# Añadir el texto con saltos de línea al dataframe
combined_pca_df['text_with_breaks'] = combined_pca_df['text'].apply(lambda x: add_line_breaks(x))

# Crear el gráfico interactivo 2D PCA
fig = px.scatter(
    combined_pca_df,
    x='PCA1',
    y='PCA2',
    color='manual',
    hover_data={'manual': True, 'text_with_breaks': True},
    labels={'manual': 'Label', 'text_with_breaks': 'Contenido'}
)

question_index = 0
# Actualizar los puntos de las preguntas para que sean de color negro y ajustar el símbolo
for trace in fig.data:
    if any(question_label in trace.name for question_label in question_labels):
        trace.marker.color = 'black'
        trace.marker.symbol = 'diamond'
        trace.name = question_labels[question_index]
        trace.hovertemplate = (
            '<b>%{customdata[0]}:</b><br>'
            '%{customdata[1]}<extra></extra>'
        )
        question_index = question_index + 1
    else:
        trace.hovertemplate = (
            '<b>Manual:</b> %{customdata[0]}<br>'
            '<b>Texto:</b> %{customdata[1]}<extra></extra>'
        )

# Aumentar el tamaño del gráfico y ajustar la caja de hover
fig.update_layout(
    title='Reducción de Dimensionalidad con PCA (2D)',
    xaxis_title='PCA1',
    yaxis_title='PCA2',
    legend_title='Label',
    width=1000,  # Ancho del gráfico
    height=700,  # Altura del gráfico
    hoverlabel=dict(
        bgcolor="white",
        font_size=12,  # Ajustar tamaño de la fuente
        font_family="Rockwell"
    )
)

fig.show()


**Conclusiones de los Gráficos de PCA (2D Y 3D)**

El análisis de reducción de dimensionalidad con PCA, tanto en 2D como en 3D, nos lleva a las mismas conclusiones clave:

No se observan agrupamientos claros entre las diferentes categorías de documentos, lo que implica que incluso al considerar hasta tres componentes principales, no se logra una separación completa de las categorías. Esto también sugiere que la variabilidad relevante para la clasificación de categorías está distribuida en más dimensiones.

Además, las preguntas etiquetadas (Q1 a Q9) están dispersas principalmente en la región central del gráfico, lo que indica que las preguntas y los fragmentos de texto asociados comparten características semánticas similares. La falta de agrupamientos claros entre las categorías y la dispersión de las preguntas indican que, aunque PCA captura una parte significativa de la variabilidad en los datos, no logra una separación clara entre las categorías de documentos.

En resumen, los análisis de PCA en 2D y 3D indican que las primeras componentes principales no son suficientes para separar claramente las categorías de docu.mentos

### Reducción de Dimensionalidad con TSNE (2D)

In [45]:
tsne_df = pd.DataFrame(tsne_result, columns=['TSNE1', 'TSNE2'])
tsne_df['manual'] = chunk_labels
tsne_df['type'] = 'chunk'
tsne_df['text'] = all_chunks  # Añadir el texto del chunk para el hover

# Agregar las preguntas al dataframe
question_tsne_df = pd.DataFrame(question_tsne_results, columns=['TSNE1', 'TSNE2'])
question_tsne_df['manual'] = question_labels
question_tsne_df['type'] = 'question'
question_tsne_df['text'] = questions  # Añadir la pregunta para el hover

# Combinar ambos dataframes
combined_tsne_df_2d = pd.concat([tsne_df, question_tsne_df])

# Añadir el texto con saltos de línea al dataframe
combined_tsne_df_2d['text_with_breaks'] = combined_pca_df['text'].apply(lambda x: add_line_breaks(x))

# Gráfico TSNE 2D
fig_tsne_2d = px.scatter(
    combined_tsne_df_2d,
    x='TSNE1',
    y='TSNE2',
    color='manual',
    hover_data={'manual': True, 'text_with_breaks': True},
    labels={'manual': 'Label', 'text_with_breaks': 'Contenido'}
)

question_index = 0
for trace in fig_tsne_2d.data:
    if any(question_label in trace.name for question_label in question_labels):
        trace.marker.color = 'black'
        trace.marker.symbol = 'diamond'
        trace.name = question_labels[question_index]
        trace.hovertemplate = (
            '<b>%{customdata[0]}:</b><br>'
            '%{customdata[1]}<extra></extra>'
        )
        question_index = question_index + 1
    else:
        trace.hovertemplate = (
            '<b>Manual:</b> %{customdata[0]}<br>'
            '<b>Texto:</b> %{customdata[1]}<extra></extra>'
        )
fig_tsne_2d.update_layout(
    title='Reducción de Dimensionalidad con TSNE (2D)',
    xaxis_title='TSNE1',
    yaxis_title='TSNE2',
    legend_title='Label',
    width=1000,
    height=700,
    hoverlabel=dict(
        bgcolor="white",
        font_size=12,
        font_family="Rockwell"
    )
)
fig_tsne_2d.show()


**Conclusiones del Gráfico de t-SNE (2D y 3D)**

En conclusión, tanto el análisis de t-SNE en 2D como en 3D muestra que no se observan agrupamientos claros entre las diferentes categorías de documentos, lo que implica que las primeras componentes t-SNE no son suficientes para separar completamente las categorías. Esto también sugiere que la variabilidad relevante para la clasificación de categorías está distribuida en más dimensiones.

Además, las preguntas etiquetadas (Q1 a Q9) están dispersas, lo que indica que las preguntas y los fragmentos de texto asociados comparten características semánticas similares. La falta de agrupamientos claros entre las categorías y la dispersión de las preguntas indican que, aunque t-SNE captura una parte significativa de la variabilidad en los datos, no logra una separación clara entre las categorías de documentos.

El análisis de reducción de dimensionalidad con t-SNE proporciona una mejor separación y agrupamiento de los datos en comparación con PCA, revelando estructuras más complejas y naturales en los datos textuales de los manuales de Sabentis. No obstante, la presencia de superposición entre algunas categorías sugiere la necesidad de un análisis más profundo para una clasificación más precisa y útil.